# The Analyzer

In [2]:
# Libraries
import json
import urllib
# import folium
import requests
import pandas as pd
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('wine_data.csv', 
                 sep=',', 
                 encoding='utf-8')

In [4]:
import requests
import urllib.parse

geolocation = []
n = 1
for i in df['land'].astype(str):
    
    print('Geocoding address', 
          n, 
          'out of', 
          len(df['land']), 
          ':', 
          i)
    n=n+1
    clear_output(wait=True)
    
    try:
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(i) +'?format=json'
        response = requests.get(url).json()
        geolocation.append(pd.Series(data={'lat': response[0]["lat"], 'lon': response[0]["lon"]}))

    except:
        geolocation.append(pd.Series(data={'lat': None, 'lon': None}))
        
        

# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df['land'])))
df['lat'] = df_loc['lat']
df['lon'] = df_loc['lon']
df.head(50)

,typ,wein_name,land,region,produzent,rebsorte,alkohol,inhalt_cl,anzahl_fl,jahrgang,prime_start,prime_end,preis_chf,preis_cl,lat,lon
0,rot,Apothic Inferno Red Blend Wine with a Whiskey ...,USA,NaN,E. & J. Gallo Winery,NaN,16.0,75.0,1,2018.0,2022.0,2023.0,19.95,0.27,39.7837304,-100.445882
1,rot,Apothic Dark California Red Blend,USA,NaN,NaN,diverse Sorten,14.0,75.0,1,2016.0,2022.0,2024.0,12.95,0.17,39.7837304,-100.445882
2,rot,Antonini Monte Chiara Montepulciano d'Abruzzo DOC,Italien,Abruzzen,Montepulciano,NaN,12.5,75.0,1,2020.0,2023.0,2024.0,3.75,0.05,42.6384261,12.674297
3,rot,Angelin Langhe Nebbiolo DOC,Italien,Piemont,Angelo Negro,Nebbiolo,14.0,75.0,1,2021.0,2022.0,2026.0,17.50,0.23,42.6384261,12.674297
4,rot,Amarone della Valpolicella DOCG Vigne Alte Zeni,Italien,Venetien,Cantina Zeni,Molinara,15.5,75.0,1,2019.0,2024.0,2028.0,32.95,0.44,42.6384261,12.674297
5,rot,Amarone della Valpolicella DOCG Vigne Alte Zeni,Italien,Venetien,Cantina Zeni,Molinara,15.5,75.0,1,2018.0,2023.0,2027.0,32.95,0.44,42.6384261,12.674297
6,rot,Amarone della Valpolicella DOCG Rocca Alata,Italien,Venetien,Cantina di Soave,Corvina,14.5,75.0,1,2018.0,2019.0,2025.0,21.50,0.29,42.6384261,12.674297
7,rot,Amarone della Valpolicella DOCG Riserva di Cos...,Italien,Venetien,Masi,Molinara,15.5,75.0,1,2016.0,2019.0,2024.0,48.50,0.65,42.6384261,12.674297
8,rot,Amarone della Valpolicella DOCG Palazzo Maffei,Italien,Venetien,Palazzo Maffei,Molinara,15.0,75.0,1,2019.0,2023.0,2026.0,24.95,0.33,42.6384261,12.674297
9,rot,Amarone della Valpolicella DOCG classico Zeni,Italien,Venetien,Cantina Zeni,Molinara,15.0,75.0,1,2019.0,2023.0,2026.0,24.95,0.33,42.6384261,12.674297


In [5]:
df.to_csv('wine_data_geo.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)